In [1]:
%load_ext autoreload
%autoreload 2


import networkx as nx
import os
import pickle
from tqdm import tqdm
import numpy as np
import random
from collections import defaultdict
import copy
from wolf import *
from run import *

In [2]:
# IC MODEL
PROPOGATION = 0.1
total_runs = 30
ITERATIONS = 100
SEED_SIZE = 25
POPULATION_SIZE = 500

In [3]:
import concurrent.futures
    
# with concurrent.futures.ThreadPoolExecutor() as executor:
#     results = executor.map(simulate, range(30))


In [4]:
attribute = "age"
objectives = {'maximin'}
demo = simulate(get_graphs(), attribute, objectives, 1, True)
demo

100%|██████████| 24/24 [03:02<00:00,  7.59s/it]


array([6.12685185e+01, 8.28682245e-02, 2.30185185e+01, 1.18440670e-02,
       6.20370370e-01, 1.13639111e+00, 5.77307450e-01, 1.04625905e+00,
       8.64701814e-01])

In [5]:
# # print(np.average(results, axis=0))
# # print(sum(results[:,2]==0)/len(results))
# # print(np.average(results2, axis=0)/(np.average(results, axis=0)))
# np.array(np.concatenate((np.average(results, axis=0),
# np.array([sum(results[:,2]==0)/len(results)]), 
# np.average(results2, axis=0)/(np.average(results, axis=0)))))
        

In [6]:
# !python wolf.py -a age -o maximin --pof